In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from scipy import stats

# Prepare data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
test_ids = test_data['id']

# Preprocess data
scaler = StandardScaler()
le = LabelEncoder()
train_data.drop_duplicates(inplace=True)

# Detect and handle outliers using Z-score method
def remove_outliers_zscore(data, threshold=3):
    z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))
    return data[(z_scores < threshold).all(axis=1)]

train_data = remove_outliers_zscore(train_data)

# Define X and y
X = train_data.drop(["Fault Type"], axis=1)
y = train_data["Fault Type"]
y = le.fit_transform(y)
y = to_categorical(y)

# Handle class imbalance with RandomOverSampler
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, np.argmax(y, axis=1))
y = to_categorical(y)

# Scale features
X = scaler.fit_transform(X)
test_data = scaler.transform(test_data)

def create_advanced_model():
    model = Sequential()
    model.add(Dense(256, input_dim=X.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define parameter grids
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 6, 9]
}

# Create RandomizedSearchCV objects
rf_random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_distributions=rf_param_grid, cv=3, n_iter=10, scoring='accuracy', verbose=2, random_state=42, n_jobs=-1)
xgb_random_search = RandomizedSearchCV(XGBClassifier(random_state=42), param_distributions=xgb_param_grid, cv=3, n_iter=10, scoring='accuracy', verbose=2, random_state=42, n_jobs=-1)

# Number of folds
n_splits = 3
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
nn_accuracies = []
rf_accuracies = []
xgb_accuracies = []
stacked_accuracies = []

for train_index, val_index in kf.split(X, np.argmax(y, axis=1)):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Neural Network
    model = create_advanced_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
    model_save = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[model_save, early_stopping, reduce_lr], verbose=0)

    # Evaluate NN model
    y_pred_proba = model.predict(X_val)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_val_labels = np.argmax(y_val, axis=1)
    nn_accuracy = accuracy_score(y_val_labels, y_pred)
    nn_accuracies.append(nn_accuracy)

    # Random Forest
    rf_random_search.fit(X_train, np.argmax(y_train, axis=1))
    best_rf_model = rf_random_search.best_estimator_
    y_pred_rf = best_rf_model.predict(X_val)
    rf_accuracy = accuracy_score(np.argmax(y_val, axis=1), y_pred_rf)
    rf_accuracies.append(rf_accuracy)

    # XGBoost
    xgb_random_search.fit(X_train, np.argmax(y_train, axis=1))
    best_xgb_model = xgb_random_search.best_estimator_
    y_pred_xgb = best_xgb_model.predict(X_val)
    xgb_accuracy = accuracy_score(np.argmax(y_val, axis=1), y_pred_xgb)
    xgb_accuracies.append(xgb_accuracy)

    # Stacked Model
    nn_predictions = model.predict(X_val)
    rf_predictions = best_rf_model.predict_proba(X_val)
    xgb_predictions = best_xgb_model.predict_proba(X_val)
    final_predictions = (nn_predictions + rf_predictions + xgb_predictions) / 3
    y_pred_stacked = np.argmax(final_predictions, axis=1)
    stacked_accuracy = accuracy_score(y_val_labels, y_pred_stacked)
    stacked_accuracies.append(stacked_accuracy)

print(f'Neural Network Accuracy: {np.mean(nn_accuracies):.4f} ± {np.std(nn_accuracies):.4f}')
print(f'Random Forest Accuracy: {np.mean(rf_accuracies):.4f} ± {np.std(rf_accuracies):.4f}')
print(f'XGBoost Accuracy: {np.mean(xgb_accuracies):.4f} ± {np.std(xgb_accuracies):.4f}')
print(f'Stacked Model Accuracy: {np.mean(stacked_accuracies):.4f} ± {np.std(stacked_accuracies):.4f}')

# Train final models on full training data
final_nn_model = create_advanced_model()
final_nn_model.fit(X, y, epochs=50, batch_size=32, verbose=2, callbacks=[early_stopping, reduce_lr])

rf_random_search.fit(X, np.argmax(y, axis=1))
final_rf_model = rf_random_search.best_estimator_

xgb_random_search.fit(X, np.argmax(y, axis=1))
final_xgb_model = xgb_random_search.best_estimator_

# Predict on test data
nn_predictions = final_nn_model.predict(test_data)
rf_predictions = final_rf_model.predict_proba(test_data)
xgb_predictions = final_xgb_model.predict_proba(test_data)

# Average the predictions
final_predictions = (nn_predictions + rf_predictions + xgb_predictions) / 3
y_pred_labels = np.argmax(final_predictions, axis=1)
y_pred_labels_inv = le.inverse_transform(y_pred_labels)

# Prepare submission DataFrame
submission = pd.DataFrame({
    'id': test_ids,
    'Fault Type': y_pred_labels_inv
})

# Save submission
submission.to_csv('Sample_sub.csv', index=False)


9/9 [==============================] - 0s 3ms/step
Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Fitting 3 folds for each of 10 candidates, totalling 30 fits
9/9 [==============================] - 0s 2ms/step
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
9/9 [==============================] - 0s 2ms/step
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
9/9 [==============================] - 0s 2ms/step
Neural Network Accuracy: 0.9495 ± 0.0047
Random Forest Accuracy: 0.9912 ± 0.0047
XGBoost Accuracy: 0.9949 ± 0.0071
Stacked Model Accuracy: 0.9975 ± 0.0036
Epoch 1/50


25/25 - 2s - loss: 2.2862 - accuracy: 0.3396 - lr: 0.0010 - 2s/epoch - 75ms/step
Epoch 2/50


25/25 - 0s - loss: 1.6131 - accuracy: 0.4697 - lr: 0.0010 - 81ms/epoch - 3ms/step
Epoch 3/50


...

25/25 - 0s - loss: 0.5024 - accuracy: 0.8990 - lr: 0.0010 - 153ms/epoch - 6ms/step
Epoch 47/50


25/25 - 0s - loss: 0.5498 - accuracy: 0.8851 - lr: 0.0010 - 169ms/epoch - 7ms/step
Epoch 48/50


25/25 - 0s - loss: 0.5025 - accuracy: 0.8813 - lr: 0.0010 - 156ms/epoch - 6ms/step
Epoch 49/50


25/25 - 0s - loss: 0.4837 - accuracy: 0.8990 - lr: 0.0010 - 145ms/epoch - 6ms/step
Epoch 50/50


25/25 - 0s - loss: 0.5221 - accuracy: 0.8838 - lr: 0.0010 - 175ms/epoch - 7ms/step
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
5/5 [==============================] - 0s 3ms/step
